# **과제. 청주시 시설관리공단 민원 챗봇 만들기**

* 청주시의 시설관리공단에 있는 FAQ를 벡터DB로 만들어 RAG를 구성해 봅시다.
* https://www.cjsisul.or.kr/home/sub.do?menukey=48

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [2]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

# !pip install -r /content/drive/MyDrive/langchain/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 2) 라이브러리 로딩

In [1]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [4]:
# def load_api_key(filepath):
#     with open(filepath, 'r') as file:
#         return file.readline().strip()

# path = '/content/drive/MyDrive/langchain/'

# # API 키 로드 및 환경변수 설정
# openai.api_key = load_api_key(path + 'api_key.txt')
# os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [2]:
print(os.environ['OPENAI_API_KEY'])

sk-svcacct-h6MQscw9uInllFfsu-J_Oj3rxcNhiswpv_Eh7dIqRBasiiddODZT3BlbkFJXZ1UnL7qMUoQPbGQ-GEpUyIgwGbjRSvE6FEvgC9qgqFl4xZ-jAA


In [6]:
path = '/content/drive/MyDrive/langchain/'

## **2.Vector DB 만들기**

* 시설관리 공단의 QA 데이터를 더 추가합니다.
    * 현재는 4건만 있음
* 추가한 csv 파일을 로딩해서 vectorDB를 구성
* 질문을 시도해 봅시다.
* 메모리를 구성해서

* 데이터 로딩

In [5]:
csv_path = 'aivle.csv'
data = pd.read_csv(csv_path, encoding='utf-8')
data.head()

,구분,QA
0,공영주차장,공영주차장의 운영시간은 어떻게 되나요?\n노상 공영주차장: 평일 09:00~18:3...
1,공영주차장,주차요금을 내고 싶은데 어떻게 해야 하나요?\n주차요금이 발부된 날로부터 10일 이...
2,청주실내빙상장,청주실내빙상장 운영시간\n일일입장 10:00~18:00 (12:00~13:00)점심...
3,청주실내빙상장,"청주실내빙상장 이용료\n입장료 : 성인 3,500원/청소년 3,000원/어린이 2,..."


* 벡터 데이터베이스
    * Embedding 모델 : text-embedding-ada-002
    * DB 경로 : path + /db3



In [3]:
db_path = '../aivle_db'
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory= db_path, embedding_function = embeddings )

* 데이터 입력
    * DF to Vector DB

In [7]:
# 각 행의 데이터를 Document 객체로 변환
documents = [Document(page_content=text) for text in data.loc[0:1, 'QA'].tolist()]

In [8]:
# 데이터프레임에서 문서 추가
database.add_documents(documents)

['8106fa9c-da08-4b64-bb8f-408fbe8afd9b',
 'd456c51a-3d32-4a13-b1ec-aac7265820bd']

* 입력된 데이터 조회

In [9]:
database.get()

{'ids': ['8106fa9c-da08-4b64-bb8f-408fbe8afd9b',
  'd456c51a-3d32-4a13-b1ec-aac7265820bd'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['공영주차장의 운영시간은 어떻게 되나요?\n노상 공영주차장: 평일 09:00~18:30(9시간 30분), 토, 일요일·공휴일 무료\n용암동 공영주차장: 평일 09:00~18:00(9시간), 토, 일요일·공휴일 무료\n도매시장 부설주차장: 평일·토요일 05:00~18:00(13시간), 일요일·공휴일 무료\n환승주차장: 24시간 연중 무휴(365일 운영)\n공영주차장 운영 시간은 변동 될 수 있음',
  '주차요금을 내고 싶은데 어떻게 해야 하나요?\n주차요금이 발부된 날로부터 10일 이내에 공단에서 운영하는 공영주차장 근무자에게 납부하는 방법\n은행계좌로 송금하는 방법(농협은행 313-01-133975)\n지로고지서 납부방법 : 전국 금융기관에서 납부가능\n인터넷, 텔레뱅킹 이용 방법 : 입금 시 입금자명 차량번호로 변경'],
 'uris': None,
 'data': None}

In [10]:
# 질문으로 유사도 검색해 보기
query = "주차요금은 어떻게 내?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


[(Document(page_content='주차요금을 내고 싶은데 어떻게 해야 하나요?\n주차요금이 발부된 날로부터 10일 이내에 공단에서 운영하는 공영주차장 근무자에게 납부하는 방법\n은행계좌로 송금하는 방법(농협은행 313-01-133975)\n지로고지서 납부방법 : 전국 금융기관에서 납부가능\n인터넷, 텔레뱅킹 이용 방법 : 입금 시 입금자명 차량번호로 변경'), 0.19700739773705325), (Document(page_content='공영주차장의 운영시간은 어떻게 되나요?\n노상 공영주차장: 평일 09:00~18:30(9시간 30분), 토, 일요일·공휴일 무료\n용암동 공영주차장: 평일 09:00~18:00(9시간), 토, 일요일·공휴일 무료\n도매시장 부설주차장: 평일·토요일 05:00~18:00(13시간), 일요일·공휴일 무료\n환승주차장: 24시간 연중 무휴(365일 운영)\n공영주차장 운영 시간은 변동 될 수 있음'), 0.30083836185410845)]
--------------------------------------------------
유사도 점수 : 0.19701, 문서 내용: 주차요금을 내고 싶은데 어떻게 해야 하나요?
주차요금이 발부된 날로부터 10일 이내에 공단에서 운영하는 공영주차장 근무자에게 납부하는 방법
은행계좌로 송금하는 방법(농협은행 313-01-133975)
지로고지서 납부방법 : 전국 금융기관에서 납부가능
인터넷, 텔레뱅킹 이용 방법 : 입금 시 입금자명 차량번호로 변경
유사도 점수 : 0.30084, 문서 내용: 공영주차장의 운영시간은 어떻게 되나요?
노상 공영주차장: 평일 09:00~18:30(9시간 30분), 토, 일요일·공휴일 무료
용암동 공영주차장: 평일 09:00~18:00(9시간), 토, 일요일·공휴일 무료
도매시장 부설주차장: 평일·토요일 05:00~18:00(13시간), 일요일·공휴일 무료
환승주차장: 24시간 연중 무휴(365일 운영)
공영주차장 운영 시간은 변동 될 수 있음


In [11]:
result[1][1]

0.30083836185410845

## **3.RAG+LLM모델**

### (1) 기본모델 구성

* 모델 : RetrievalQA
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정

* database를 retriever로 선언하기

In [12]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

* 모델 선언

In [13]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

* 모델 확인

In [14]:
query = "주차요금은 어떻게 내?"
result = qa(query)

print(result["result"])

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


주차요금을 내는 방법은 다음과 같습니다:
1. 공영주차장 근무자에게 직접 납부: 주차요금이 발부된 날로부터 10일 이내에 공영주차장 근무자에게 납부할 수 있습니다.
2. 은행계좌로 송금: 농협은행 313-01-133975 계좌로 송금할 수 있습니다.
3. 지로고지서 납부: 전국 금융기관에서 납부 가능합니다.
4. 인터넷 뱅킹 이용: 입금 시 입금자명을 차량번호로 변경하여 납부할 수 있습니다.

원하는 방법을 선택하여 주차요금을 납부하시면 됩니다.


### (2) 메모리관리 모델 생성

* 모델 선언

In [15]:
# 모델 선언

chat = ChatOpenAI(model="gpt-3.5-turbo")

# Retriever 지정
k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question",
                                  output_key="answer", return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")



* 모델 확인

In [16]:
query = "주차는 어떻게 해?"
result = qa(query)


print(result["answer"])

Number of requested results 3 is greater than number of elements in index 2, updating n_results = 2


주차요금을 내고 싶으시다면, 주차요금이 발부된 날로부터 10일 이내에 공단에서 운영하는 공영주차장 근무자에게 납부하거나 은행계좌로 송금하실 수 있습니다. 또한, 지로고지서를 통해 전국의 금융기관에서 납부하거나 인터넷 뱅킹, 텔레뱅킹을 통해 입금하실 수도 있습니다. 입금 시에는 입금자명을 차량번호로 기입하시면 됩니다. 

또한, 공영주차장의 운영시간은 공영주차장마다 다를 수 있지만, 대체적으로 주중에는 09:00부터 오후 6시까지 운영되며, 주말 및 공휴일에는 무료로 이용할 수 있습니다. 환승주차장은 연중무휴로 24시간 운영됩니다. 다만, 공영주차장의 운영시간은 변동될 수 있으니 주차장을 이용하기 전에 운영시간을 확인하시는 것이 좋습니다.
